# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [13]:
n_samples = 5

In [ ]:
# Verify that loading works, for one example
counterfact_dataset = CounterFact(model, split="train", nrows=n_samples)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=2)

clean_input, corrupted_input, labels, paraphrase_prompts, random_prompts = next(iter(counterfact_dataloader))

print(clean_input)
print(corrupted_input)
print(labels)
print(paraphrase_prompts)
print(random_prompts)


['The mother tongue of Danielle Darrieux is', 'The official religion of Edwin of Northumbria is']
['The mother tongue of Paul McCartney is', 'The official religion of Rasul Gamzatov is']
tensor([[24111, 15823],
        [20298, 16991]], device='cuda:0')
(['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native', 'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'], ['The school chiefly served tribal girls of Dang. Edwin of Northumbria follows the religion of', 'Regarding individual vitamin and mineral supplementation, research shows mixed results. Edwin of Northumbria is affiliated with the religion'])
(['SKF is located in', 'Köping Municipality, in', 'Upplands Väsby, in', 'Motala, in', 'Trollhättan, in', 'Upplands Väsby is located in the country of', 'Kungsör Municipality, located in', 'IKEA, located in', 'Täby, located in', 'IKEA, which is located in', 'Robert Radecke performs on the', 'Peter Igelhoff, the', 'Gre

In [14]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model, split="train", nrows=n_samples)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels, paraphrase_prompts, random_prompts = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

In [15]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

torch.cuda.empty_cache()

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model, split="train", nrows=n_samples)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels, paraphrased, random) in enumerate(counterfact_dataloader):

    paraphrased = paraphrased[0]
    random = random[0][:5]

    original_output = model.generate(clean_input, max_new_tokens=3, do_sample=False)

    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, paraphrased, random, target_mlp[n], target_attn[n])

    print("Edited output:", edited_model.generate(clean_input, max_new_tokens=3, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(edited_model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    del edited_model
    torch.cuda.empty_cache()

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

100%|██████████| 3/3 [00:00<00:00, 38.48it/s]


Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother tongue of Danielle Darrieux is French.


Fine tuning model...
Target MLP tensor(1494, device='cuda:0')
Target attn tensor(8, device='cuda:0')
Epoch 0/5, Loss: (11.486885070800781, 13.580451011657715, 8.118217468261719)
Epoch 1/5, Loss: (7.64299201965332, 9.057137489318848, 7.22287654876709)
Epoch 2/5, Loss: (5.165627479553223, 5.853238105773926, 6.34792423248291)
Epoch 3/5, Loss: (2.5930721759796143, 3.3968725204467773, 5.459205627441406)
Epoch 4/5, Loss: (0.2693476378917694, 1.0653750896453857, 4.462160587310791)


100%|██████████| 3/3 [00:00<00:00, 38.24it/s]


Edited output: The mother tongue of Danielle Darrieux isEnglishEnglishEnglish
Original label: French
Target label: English


100%|██████████| 5/5 [00:00<00:00, 38.11it/s]


...

100%|██████████| 3/3 [00:00<00:00, 38.66it/s]


Edited output: What is the twin city of Lyon? It isManManMan
Original label: Beirut
Target label: Manila


100%|██████████| 5/5 [00:00<00:00, 38.42it/s]


Generated Outputs: ["Lyon's twin city is known forManManManManMan", "Lyon's twin city is known forManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "People in Lyon's twin city speak the language ofManManManManMan", "Lyon's twin city is known forManManManManMan", "Lyon's twin city has famous tourist attractions includingManManManManMan", "Lyon's twin city has famous tourist attractions includingManManManManMan", "Lyon's twin city has famous tourist attractions includingManManManManMan"]


100%|██████████| 5/5 [00:00<00:00, 35.39it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [16]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.018234,1.0,0.422506,1.0,0.013601,1.0,0.013601
1,1.0,0.999999,1.0,0.999998,1.0,0.999998,1.0,0.999998
2,1.0,0.998850,1.0,0.999850,1.0,0.997462,1.0,0.997462
3,1.0,0.696095,1.0,0.998444,1.0,0.957632,1.0,0.957632
4,1.0,0.701500,1.0,0.503968,1.0,0.206080,1.0,0.206080
